# Bab 17: Autoencoders and GANs (Autoencoder dan GAN)

### 1. Pendahuluan

Bab 17 membahas dua keluarga arsitektur *Neural Network* yang sangat menarik dan kuat dalam kategori *unsupervised learning* atau *generative models*: **Autoencoders** dan **Generative Adversarial Networks (GANs)**. Kedua jenis model ini berfokus pada pembelajaran representasi data (*representation learning*) dan/atau generasi data baru yang realistis (*generative learning*).

---

### 2. Autoencoders (Autoencoder)

*Autoencoder* adalah Jaringan Saraf Tiruan yang dilatih untuk menghasilkan output yang hampir identik dengan inputnya. Ini mungkin terdengar tidak berguna, tetapi *Autoencoder* tidak hanya sekadar menyalin; mereka dilatih untuk melakukannya di bawah beberapa kendala, yang memaksa mereka untuk mempelajari representasi data yang efisien.

#### a. Arsitektur Autoencoder
Sebuah *Autoencoder* umumnya terdiri dari dua bagian:
* **Encoder:** Memetakan data input ke representasi berdimensi lebih rendah (disebut *codings* atau *latent representation*).
* **Decoder:** Merekonstruksi data asli dari *codings*.

Kendala yang diterapkan (misalnya, dimensi *coding* yang lebih rendah) mencegah *Autoencoder* untuk hanya menjiplak input, memaksanya untuk belajar fitur-fitur penting.

#### b. Stacked Autoencoders (Autoencoder Bertumpuk)
*Autoencoder* dengan beberapa *hidden layer* disebut *Stacked Autoencoder*. Menambahkan lebih banyak lapisan membantu model mempelajari representasi yang lebih kompleks.


In [1]:
import tensorflow as tf
from tensorflow import keras

# Membangun Stacked Autoencoder untuk Fashion MNIST
stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"), # Lapisan codings
])

stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

# Mengompilasi dengan loss binary_crossentropy karena output piksel antara 0 dan 1
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.5))

# (Kode untuk memuat data dan melatih model akan mengikuti pola yang sama
# seperti pada bab-bab sebelumnya)

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### c. Denoising Autoencoders (Autoencoder Penghilang Noise)
Dengan menambahkan noise pada input dan melatih model untuk merekonstruksi input asli yang bersih, Denoising Autoencoder dipaksa untuk mempelajari fitur-fitur yang lebih robust.

In [3]:
# Menambahkan lapisan Dropout pada input untuk simulasi noise
denoising_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),  # Coding layer to match decoder input
])

denoising_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    # ...
])

denoising_ae = keras.models.Sequential([denoising_encoder, denoising_decoder])

### d. Variational Autoencoders (VAE)
VAE adalah generative autoencoder, yang berarti selain dapat mengompresi dan merekonstruksi data, ia juga dapat menghasilkan sampel baru yang terlihat seperti data pelatihan. VAE melakukannya dengan mempelajari distribusi probabilitas dari data laten, bukan hanya satu titik coding.

---

### 3. Generative Adversarial Networks (GANs)
GANs adalah pendekatan yang sangat berbeda untuk pemodelan generatif. Arsitektur ini terdiri dari dua jaringan yang "bersaing" satu sama lain:

* **Generator**: Bertugas menghasilkan data palsu (misalnya, gambar) yang terlihat serealistis mungkin.
* **Discriminator**: Bertugas membedakan antara data asli dan data palsu yang dihasilkan oleh generator.
Selama pelatihan, keduanya menjadi lebih baik dalam tugas mereka masing-masing. Generator belajar menghasilkan gambar yang semakin sulit dibedakan, sementara discriminator menjadi semakin ahli dalam mendeteksi kepalsuan. Keseimbangan ini (disebut Nash equilibrium) menghasilkan generator yang mampu membuat data yang sangat realistis.

In [4]:
# Kerangka implementasi GAN sederhana untuk Fashion MNIST
codings_size = 30

# Generator
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

# Discriminator
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

# Model GAN yang menggabungkan keduanya
gan = keras.models.Sequential([generator, discriminator])

# Mengompilasi model
# Discriminator dilatih secara terpisah
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
# Generator dilatih melalui model GAN, di mana discriminator dibekukan
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

### a. Tantangan Pelatihan GAN
Pelatihan GAN terkenal sulit karena menemukan Nash equilibrium antara generator dan discriminator sangatlah rumit. Jika salah satu menjadi terlalu kuat, pelatihan bisa gagal.

### b. Deep Convolutional GAN (DCGAN)
Untuk menghasilkan gambar yang lebih realistis dan beresolusi lebih tinggi, digunakan arsitektur DCGAN. Kunci utamanya adalah menggunakan lapisan konvolusional pada generator (disebut transposed convolutions untuk upsampling) dan discriminator.